We'll create a folder ``chessboard`` where we'll place images of the 6x9 chessboard pattern.

In [1]:
import os

image_dir = 'chessboard'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(image_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


### Display live camera feed

So let's get started.  First, let's initialize and display our camera.
> The SSD Network takes a 300x300 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset.

In [2]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image = widgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


Next, let's create and display some buttons that we'll use to save snapshots of the chessboard pattern.
We'll also add some text boxes that will display how many images we've collected so far. 

In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
image_button = widgets.Button(description='Add Image', button_style='success', layout=button_layout)
image_count = widgets.IntText(layout=button_layout, value=len(os.listdir(image_dir)))



In [4]:
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

In [4]:
from uuid import uuid1

def save_snapshot(directory, i):
    image_path = os.path.join(directory, 'pattern'+str(i+1)+'.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_image():
    global image_dir,image_count
    save_snapshot(image_dir, image_count.value)
    image_count.value = len(os.listdir(image_dir))

    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
image_button.on_click(lambda x: save_image())

Great! Now the buttons above should save images to the ``chessboard`` directories.  You can use the Jupyter Lab file browser to view these files!

Now go ahead and collect some data 

1. Place the chessboard pattern in different orientations in front of the camera and press ``add image`
2. Repeat 1

> REMINDER: You can move the widgets to new windows by right clicking the cell and clicking ``Create New View for Output``.  Or, you can just re-display them
> together as we will below


In [5]:
display(image)
display(widgets.HBox([image_count, image_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [6]:
camera.unobserve_all()
time.sleep(1.0)

NameError: name 'time' is not defined

You should see a file named ``dataset.zip`` in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting ``Download``.

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the *host*) to train the collision avoidance neural network.  We'll assume that you've set up your training
machine as described in the JetBot WiKi.  If you have, you can navigate to ``http://<host_ip_address>:8888`` to open up the Jupyter Lab environment running on the host.  The notebook you'll need to open there is called ``collision_avoidance/train_model.ipynb``.

So head on over to your training machine and follow the instructions there!  Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!